### Decorators 2

Мы увидели, как создавать некоторые простые и не очень простые декораторы.

Однако мы также использовали встроенные декораторы, которые могут принимать параметры, такие как `wraps` и `lru_cache`.

Это может быть весьма полезно, и мы можем сделать то же самое самостоятельно.

Сначала вспомним наш оригинальный декоратор таймера:

In [1]:
def timed(fn):
    from time import perf_counter

    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        print('Run time: {0:.6f}s'.format(elapsed))
        return result

    return inner

In [2]:
def calc_fib_recurse(n):
    return 1 if n < 3 else calc_fib_recurse(n-1) + calc_fib_recurse(n-2)

def fib(n):
    return calc_fib_recurse(n)

Мы можем украсить нашу функцию Фибоначчи, используя синтаксис **@** или более длинный синтаксис следующим образом:

In [3]:
fib = timed(fib)

In [4]:
fib(30)

Run time: 0.255260s


832040

Давайте изменим это так, чтобы таймер запускал функцию несколько раз и вычислял среднее время выполнения:

In [5]:
def timed(fn):
    from time import perf_counter

    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(10):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (perf_counter() - start)
        avg_elapsed = total_elapsed / 10
        print('Avg Run time: {0:.6f}s'.format(avg_elapsed))
        return result

    return inner

И снова мы декорируем его, используя длинный синтаксис:

In [6]:
def fib(n):
    return calc_fib_recurse(n)

fib = timed(fib)

In [7]:
fib(28)

Avg Run time: 0.098860s


317811

Но это значение 10 было жестко закодировано. Давайте сделаем его параметром.

In [8]:
def timed(fn, num_reps):
    from time import perf_counter

    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(num_reps):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (perf_counter() - start)
        avg_elapsed = total_elapsed / num_reps
        print('Avg Run time: {0:.6f}s ({1} reps)'.format(avg_elapsed,
                                                        num_reps))
        return result

    return inner

Теперь, чтобы decorate нашу функцию Фибоначчи, нам **придется** использовать длинный синтаксис (как мы видели на лекции, синтаксис **@** работать не будет):

In [9]:
def fib(n):
    return calc_fib_recurse(n)

fib = timed(fib, 5)

In [10]:
fib(28)

Avg Run time: 0.095708s (5 reps)


317811

Проблема в том, что мы не можем использовать синтаксис декоратора `@`, потому что при использовании этого синтаксиса Python передает декоратору **один** аргумент: декорируемую нами функцию и ничего больше.

Конечно, мы могли бы просто использовать то, что сделали выше, но синтаксис декоратора довольно аккуратен, поэтому было бы неплохо сохранить возможность его использования.

Нам просто нужно немного изменить наше мышление, чтобы сделать это:

Во-первых, когда мы видим следующий синтаксис:

`
@dec
def my_func():
pass
`

мы видим, что `dec` должна быть функцией, которая принимает один аргумент, функция декорируется.

Вы заметите, что `dec` — это просто функция, но мы не **вызываем** `dec`, когда декорируем `my_func`, мы просто используем метку `dec`.

Затем Python делает:

`
my_func = dec(my_func)
`

Давайте рассмотрим конкретный пример:

In [11]:
def dec(fn):
    print ("running dec")

    def inner(*args, **kwargs):
        print("running inner")
        return fn(*args, **kwargs)

    return inner

In [12]:
@dec
def my_func():
    print('running my_func')

running dec


Как мы видим, когда мы декорировали `my_func`, в это время была **вызвана** функция `dec`.

(Потому что Python сделал это:

`my_func = dec(my_func)`

поэтому была вызвана `dec`)

И когда мы теперь вызываем `my_func`, мы видим, что вызывается `inner` функция, за которой следует исходная `my_func`.

In [13]:
my_func()

running inner
running my_func


Но что, если `dec` не был самим декоратором, а вместо этого создавал и возвращал декоратор?

Давайте посмотрим, как мы могли бы это сделать:

In [14]:
def dec_factory():
    print('running dec_factory')
    def dec(fn):
        print('running dec')
        def inner(*args, **kwargs):
            print('running inner')
            return fn(*args, **kwargs)
        return inner
    return dec

Как видите, вызов `dec_generator()` вернет функцию `dec`, которая является нашим декоратором:

In [15]:
@dec_factory()
def my_func(a, b):
    print(a, b)

running dec_factory
running dec


Вы можете видеть, что и `dec_generator`, и `dec` уже были вызваны.

In [16]:
my_func(10, 20)

running inner
10 20


Вот и все, все, что мы сделали, это по сути создали декоратор, вызвав функцию (`dec_factory`) и использовав возвращаемое значение этого вызова (функцию `dec`) в качестве нашего фактического декоратора.

Мы могли бы сделать украшение и таким образом:

In [17]:
dec = dec_factory()

running dec_factory


In [18]:
@dec
def my_func():
    print('running my_func')

running dec


In [19]:
my_func()

running inner
running my_func


Или даже так:

In [20]:
dec = dec_factory()

def my_func():
    print('running my_func')

my_func = dec(my_func)

running dec_factory
running dec


In [21]:
my_func()

running inner
running my_func


Конечно, мы могли бы даже украсить это таким образом, используя одно выражение:

In [22]:
def my_func():
    print('running my_func')

my_func = dec_factory()(my_func)

running dec_factory
running dec


In [23]:
my_func()

running inner
running my_func


Итак, теперь мы оформили нашу функцию, используя не декоратор, а фабрику декораторов следующим образом:

In [24]:
def dec_factory():
    def dec(fn):
        def inner(*args, **kwargs):
            print('running decorator inner')
            return fn(*args, **kwargs)
        return inner
    return dec

In [25]:
@dec_factory()
def my_func(a, b):
    return a + b

In [26]:
my_func(10, 20)

running decorator inner


30

Обратите внимание, что при таком подходе мы **вызываем** `dec_factory()`, [обратите внимание на скобки `()`], а **затем** используем возвращаемое значение (декоратор) для декорирования нашей функции.

Итак, мы могли бы передавать аргументы, как мы это делаем, не влияя на конечный результат. Фактически, мы даже можем получить к ним доступ из любого места внутри `dec_factory`, включая любую из вложенных функций!

Давайте попробуем это:

In [27]:
def dec_factory(a, b):
    def dec(fn):
        def inner(*args, **kwargs):
            print('running decorator inner')
            print('free vars: ', a, b)  # a and b are free variables!
            return fn(*args, **kwargs)
        return inner
    return dec

In [28]:
@dec_factory(10, 20)
def my_func():
    print('python rocks')

In [29]:
my_func()

running decorator inner
free vars:  10 20
python rocks


И вот как мы можем создавать декораторы с параметрами. Мы не создаем декоратор напрямую, вместо этого мы используем внешнюю функцию, которая возвращает декоратор при вызове, и передаем аргументы этой внешней функции, к которым декоратор и его внутренняя функция, конечно, могут обращаться как к нелокальным (свободным) переменным.

Итак, давайте вернемся к нашей изначальной проблеме, где мы хотели, чтобы наш декоратор времени запускал несколько циклов, которые можно было бы указать в качестве параметра при декорировании функции, которую мы хотим хронометрировать.

Вот она снова:

In [30]:
def timed(fn, num_reps):
    from time import perf_counter

    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(num_reps):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (perf_counter() - start)
        avg_elapsed = total_elapsed / num_reps
        print('Avg Run time: {0:.6f}s ({1} reps)'.format(avg_elapsed,
                                                        num_reps))
        return result

    return inner

Итак, все, что нам нужно сделать, это создать внешнюю функцию вокруг нашего хронометрированного декоратора и вместо этого передать аргумент `num_reps` этой внешней функции:

In [31]:
def timed_factory(num_reps=1):
    def timed(fn):
        from time import perf_counter

        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(num_reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                total_elapsed += (perf_counter() - start)
            avg_elapsed = total_elapsed / num_reps
            print('Avg Run time: {0:.6f}s ({1} reps)'.format(avg_elapsed,
                                                            num_reps))
            return result
        return inner
    return timed

In [32]:
@timed_factory(5)
def fib(n):
    return calc_fib_recurse(n)

In [33]:
fib(30)

Avg Run time: 0.249934s (5 reps)


832040

Чтобы завершить это, мы, вероятно, не хотим, чтобы наша внешняя функция была названа так, как она есть (`timed_factory`). Вместо этого мы, вероятно, просто захотим назвать ее `timed`. Так что давайте просто сделаем эту последнюю часть:

In [34]:
from functools import wraps

def timed(num_reps=1):
    def decorator(fn):
        from time import perf_counter

        @wraps(fn)
        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(num_reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                total_elapsed += (perf_counter() - start)
            avg_elapsed = total_elapsed / num_reps
            print('Avg Run time: {0:.6f}s ({1} reps)'.format(avg_elapsed,
                                                            num_reps))
            return result
        return inner
    return decorator

In [35]:
@timed(5)
def fib(n):
    return calc_fib_recurse(n)

In [36]:
fib(30)

Avg Run time: 0.253744s (5 reps)


832040